# Pulse propagation in gas-filled hollow core fiber (HCF)

## Manual fitting with spectrum

In [1]:
# notebook.ipynb
from libraries2 import *
from variables2 import *
from functions2 import *

frequency_FWHM = wavelength_FWHM_nm * speedoflight_nmpfs / wavelength0_nm**2
duration_FWHM_fs = 0.44 /  frequency_FWHM
duration_FWHM = duration_FWHM_fs * 1e-15
wavelength_frontend_nm = np.arange(730, 1331, 1)
spectrum_amplitude_frontend = getGaussianWavelengthSpectrum(wavelength_frontend_nm, wavelength0_nm, wavelength_FWHM_nm)
I_frontend = getPower(spectrum_amplitude_frontend)
wavelength0_m = wavelength0_nm * 1e-9
wavelength_frontend_m = wavelength_frontend_nm * 1e-9
wavelength_FWHM_m = wavelength_FWHM_nm * 1e-9
frequency0 = speedoflight_mps/wavelength0_m                       
omega0 = 2*np.pi*frequency0
pulse_energy = average_power_in/repetition_frequency             
peak_power = pulse_energy/duration_FWHM                       
amplitude = np.sqrt(peak_power)
attenuation = (1 / z) * np.log(average_power_in/average_power_out)
refractive_index = calculate_refractive_index(pressure)
beta0 = refractive_index * (omega0 / speedoflight_mps)
gamma_calculated = calculate_gamma(n2_atm, pressure, wavelength0_m, core_diameter) 
dz = z / nsteps
Time_window = 50 * duration_FWHM_fs * 1e-15
t = np.linspace(-Time_window/2,Time_window/2,N)                                                                                  
dt = abs(t[1] - t[0])                                   
f = fftshift(fftfreq(N,d=dt))
f_rel = f + frequency0
omega_rel = 2 * np.pi * f_rel
wavelength_rel = speedoflight_mps / f_rel
sort_idx = np.argsort(wavelength_rel)
wavelength_rel = wavelength_rel[sort_idx]
inv_jacobian = (speedoflight_mps) / (wavelength_rel**2)
jacobian = (wavelength_rel**2) / (speedoflight_mps)
interp_func = interp1d(wavelength_frontend_m, I_frontend, kind='cubic', fill_value=0, bounds_error=False)
I_frontend_simgrid = interp_func(wavelength_rel)
I_frontend_simgrid /= np.max(I_frontend_simgrid)
#I_frontend_simgrid = np.clip(I_frontend_simgrid, 0, None)
data = np.loadtxt(filepath, delimiter=";", skiprows=1)
wavelength_spec_ref_nm = data[:, 0]
wavelength_spec_ref_m = wavelength_spec_ref_nm * 1e-9
I_spec_ref = data[:, 2]
interp_func = interp1d(wavelength_spec_ref_m, I_spec_ref, kind='cubic', fill_value=0, bounds_error=False)
I_spec_ref_simgrid = interp_func(wavelength_rel)
I_spec_ref_simgrid /= np.max(I_spec_ref_simgrid)

#pulse_init = getGaussianPulseTime(amplitude, t, duration_FWHM_fs  * 1e-15) 
A_frontend_wavelength = getGaussianWavelengthSpectrum(wavelength_rel, wavelength0_m, wavelength_FWHM_nm * 1e-9)
I_frontend_wavelength = getPower(A_frontend_wavelength)
I_frontend_frequency = A_frontend_wavelength * inv_jacobian
A_frontend_frequency = np.sqrt(I_frontend_frequency) * np.exp(1j * 0)
A_frontend = getPulseFromSpectrum(t,A_frontend_frequency)
A_frontend /= np.max(A_frontend)
pulse_init = amplitude * A_frontend

# --- Global state to keep buttons alive ---
buttons = []

# Create figure and axes
fig, ax = plt.subplots()
plt.subplots_adjust(left=0.25, bottom=0.55)  # Extra space for sliders

# Initial parameter values
gamma_init = gamma_calculated
beta2_init = beta2
f_R_init = f_R        
 
def SSFM(gammavariable, beta2variable, f_R_variable):
    A = pulse_init.copy()
    f_R, hR = raman_response(t)
    dispersion_step = np.exp(1j * (beta2variable / 2) * omega_rel**2 * dz)
    loss_step = np.exp(-(attenuation / 2) * dz)
    for n in range(nsteps):
        # Nonlinearity half-step for Kerr
        I = getPower(A)
        SPM_half_step = np.exp(1j * gammavariable * I * dz / 2)
        A *= SPM_half_step
        #  Nonlinearity half-step for Raman
        # Apply 1D convolution along time axis for all (x, y)
        raman_conv = convolve1d(I, hR)
        raman_factor = (1 - f_R_variable) * I + f_R_variable * raman_conv * dt
        Raman_half_step = np.exp(1j * gammavariable * raman_factor * dz / 2)
        A *= Raman_half_step
        # Linearity full-step for Dispersion and Loss
        A_fft = fftshift(fft(ifftshift(A)))
        A_fft *= dispersion_step * loss_step
        A = fftshift(ifft(ifftshift(A_fft)))
        # Nonlinearity half-step for Kerr
        I = getPower(A)
        SPM_half_step = np.exp(1j * gammavariable * I * dz / 2)
        A *= SPM_half_step
        #  Nonlinearity half-step for Raman
        # Apply 1D convolution along time axis for all (x, y)
        raman_conv = convolve1d(I, hR)
        raman_factor = (1 - f_R) * I + f_R * raman_conv * dt
        Raman_half_step = np.exp(1j * gammavariable * raman_factor * dz / 2)
        A *= Raman_half_step

    A_final = A
    A_spectrum_final = getSpectrumFromPulse(t, A_final)
    I_sim = getPower(A_spectrum_final)
    I_sim /= np.max(I_sim)
    I_sim *= jacobian
    I_sim /= np.max(I_sim)
    print(f"gamma = {gammavariable:.5e}, beta2 = {beta2variable:.2e}, f_R = {f_R_variable}")
    return I_sim

# --- Initial plot ---
spectrum = SSFM(gamma_init, beta2_init, f_R_init)
l, = ax.plot(wavelength_rel * 1e9, spectrum, label='Simulated')
l2, = ax.plot(wavelength_rel * 1e9, I_spec_ref_simgrid, '--', label='Measured')

# R² text placeholder
r2_text = ax.text(0.02, 0.95, '', transform=ax.transAxes, fontsize=10, verticalalignment='top')

ax.set_xlabel("Wavelength [nm]")
ax.set_ylabel("Normalized Intensity")
ax.set_title("Laser pulse propagation in gas-filled HCF")
ax.grid(True)
ax.set_ylim(0, 1)
ax.set_xlim(900, 1200)
ax.legend()

# --- Slider axes ---
ax_log_gamma = plt.axes([0.25, 0.40, 0.65, 0.03])
ax_beta2 = plt.axes([0.25, 0.30, 0.65, 0.03])
ax_f_R = plt.axes([0.25, 0.20, 0.65, 0.03])

# --- Sliders ---
slider_log_gamma = Slider(ax_log_gamma, 'log₁₀(γ)', np.log10(gamma_init), np.log10(1e-1), valinit=np.log10(gamma_init), valstep=0.01)
slider_beta2 = Slider(ax_beta2, 'beta2', -1e-26, 1e-26, valinit=beta2_init, valstep=1e-28)
slider_f_R = Slider(ax_f_R, 'f_R', 0, 1, valinit=f_R_init, valstep=0.01)

def add_arrow_buttons(y_pos, slider, step=None):
    if step is None:
        step = slider.valstep or 0.01

    ax_left = plt.axes([0.05, y_pos, 0.04, 0.03])
    ax_right = plt.axes([0.10, y_pos, 0.04, 0.03])
    btn_left = Button(ax_left, '←')
    btn_right = Button(ax_right, '→')

    def left(event):
        val = slider.val - step
        val = max(val, slider.valmin)
        slider.set_val(val)

    def right(event):
        val = slider.val + step
        val = min(val, slider.valmax)
        slider.set_val(val)

    btn_left.on_clicked(left)
    btn_right.on_clicked(right)

    # Optional: label text for context (e.g., 'α')
    ax_label = plt.axes([0.15, y_pos, 0.05, 0.03])
    ax_label.axis("off")

# --- Update function ---
def update(val):
    gammaval = 10 ** slider_log_gamma.val
    beta2val = slider_beta2.val
    f_R_val = slider_f_R.val

    # Run simulation
    new_spec = SSFM(gammaval, beta2val, f_R_val)
    l.set_ydata(new_spec)

    # Compute R²
    y_true = I_spec_ref_simgrid
    y_pred = new_spec
    ss_res = np.sum((y_true - y_pred) ** 2)
    ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    # Update title and R² text
    ax.set_title(f"gamma = {gammaval:.2e}, beta2 = {beta2val:.2e}, f_R = {f_R_val}")
    r2_text.set_text(f"R² = {r_squared:.5f}")
    fig.canvas.draw_idle()

# --- Add arrow buttons to left side ---
def add_arrow_buttons(y_pos, slider, step=None):
    if step is None:
        step = slider.valstep or 0.01

    # Create left arrow button
    ax_left = plt.axes([0.05, y_pos, 0.04, 0.03])
    btn_left = Button(ax_left, '←')
    
    def left_click(event):
        new_val = max(slider.val - step, slider.valmin)
        slider.set_val(new_val)

    btn_left.on_clicked(left_click)
    buttons.append(btn_left)

    # Create right arrow button
    ax_right = plt.axes([0.10, y_pos, 0.04, 0.03])
    btn_right = Button(ax_right, '→')

    def right_click(event):
        new_val = min(slider.val + step, slider.valmax)
        slider.set_val(new_val)

    btn_right.on_clicked(right_click)
    buttons.append(btn_right)

# Connect sliders
slider_log_gamma.on_changed(update)
slider_beta2.on_changed(update)
slider_f_R.on_changed(update)

# --- Add arrow buttons to each slider ---
add_arrow_buttons(0.40, slider_log_gamma, step=0.001)
add_arrow_buttons(0.30, slider_beta2, step=1e-29)
add_arrow_buttons(0.20, slider_f_R, step=0.001)

# Show plot
plt.show()
# === END SLIDER UI WITH R² ===

gamma = 1.83970e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 1.88255e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 1.92640e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 2.06418e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 2.31605e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 2.72112e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 3.50548e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 4.72876e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 5.81765e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 6.99435e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 7.49458e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 7.84779e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 8.03059e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 8.40906e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 8.60493e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 8.80537e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 9.01047e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 9.43512e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 9.65489e-09, beta2 = 0.00e+00, f_R = 0.18
gamma = 9.43512e-09, beta2 = 0.00e+00, f_R = 0.18
